<a href="https://colab.research.google.com/github/Hyun-Jun-Lee/Review_Sentiment_Analysis/blob/main/Movie_Review_Seniment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [ ]:
!pip install konlpy

In [2]:
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt


# Read Data

In [10]:
df = pd.read_csv('/content/drive/MyDrive/crawling_review.csv')
stopwords = pd.read_csv('/content/drive/MyDrive/stopword.csv')
stopwords = list(stopwords['stopdword'].values)

In [11]:
df.sample(10)

,Unnamed: 0,star,comment,label
8338,8839,6,유오성 오대환 연기 진짜 잘하심장혁 연기 정말 잘하는 배우인데 여기서는 뭔가느낌없어...,0
8280,8781,1,고양이바퀴벌레 생태계파괴,0
66,73,8,빌리 홀리데이의 인생과 그의 뒷조사와 조작을 했던 미정부의 탄압,1
7579,8061,6,잼없음ㅜㅜ지겨움첨부터끝까지우울한음악,0
9196,9710,2,내용이 이해가 잘 안되고 스토리가 뻔함,0
629,672,10,신이 말씀하셨습니다 너를 타임슬립시켜줄테니 마음껏해라로또를 사도 좋고땅을 미리 사놔...,1
5167,5540,10,재밋게 봤어요 잔잔한 재미도 있고 델마와 루이스가 생각나는 느낌,1
8031,8526,1,아놔 엄마새키 ㅡㅡ,0
1197,1295,10,영화가 끝나고도 영상목소리가 계속 찡하게 울려 퍼진다처음 태일이를 보았을 땐 태일이...,1
3613,3883,10,기대없이 봤는데 엄청 잼있었어요 ㅎ,1


In [12]:
stopwords[:10]

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라']

# 토큰화 및 stopword 제거

In [8]:
# 한국 형태소 분석기
okt = Okt()

In [ ]:
data = []
for sentence in tqdm(df['comment']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    data.append(stopwords_removed_sentence)